In [ ]:
from argparse import Namespace
from PIL import Image
from pathlib import Path
from utils import common
import os
from IPython.utils import io

from IPython.display import HTML, Javascript, display
import ipywidgets as wi
import ipycanvas as ca
from PIL import Image
from ipyfilechooser import FileChooser
from nokogiri.spylus import spylus
import numpy as np


from options.test_options import TestOptions
from configs import data_configs
from datasets.images_dataset import ImagesDataset
from scripts.inference import run_on_batch

import torch
import torchvision
from models.psp import pSp

from typing import *
import ipywidgets as wi
import pickle
from random import Random
from pathlib import Path
from nokogiri.working_dir import working_dir
with working_dir("/home/natsuki/stylegan2-ada-pytorch"):
    from script_util import root, fname2id
from pyzbar import pyzbar
PX = 128

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
root = Path("/home/natsuki/encavgsim_1632393929")
# encavg_1631706221 エンコード
epoch = "iteration_495000.pt"
#epoch = "best_model.pt"
ckpt = torch.load(root/f"checkpoints/{epoch}", map_location='cpu')
opts = ckpt['opts']
test_opts = TestOptions().parser.parse_args(
f"""
--exp_dir={root} \
--checkpoint_path={root}/checkpoints/{epoch} \
--data_path=/data/natsuki/whitechest_sim_val \
--test_batch_size=1 \
--test_workers=1 \
--latent_mask=10,11,12,13,14,15
""".split())
#  --latent_mask=14,15
# 15 -> 14,15 で色がガラッと変わる
opts.update(vars(test_opts))
if 'learn_in_w' not in opts:
    opts['learn_in_w'] = False
if 'output_size' not in opts:
    opts['output_size'] = 1024
opts = Namespace(**opts)

In [ ]:
print("loading please wait...", end="")
with io.capture_output() as captured:
    net = pSp(opts)
    net.eval()
    net.cuda();
print("model loaded!")

In [ ]:
print("ok")
def mapping(G, seed=1, psi=1):
    label = torch.zeros([1, G.c_dim], device="cuda")
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).cuda()
    w = G.mapping(z, label, truncation_psi=psi)
    return w 

In [ ]:
def load_wimage(
    fname: str,
    PX: int = PX,
    ):
    try:
        with open(fname, "rb") as f:
            buf = f.read()
        image = wi.Image(value=buf, format='png', width=PX, height=PX)
    except Exception as e:
        image = wi.Image(width=PX, height=PX)
    return image
class Pannel(wi.VBox):
    def on_click(self, event) -> None:
        self.bag.clear()
        self.c(self.fname)
        self.bag.add(self.fname)
        self.output.clear_output()
        with self.output:
            print(self.id)
    def __init__(
        self,
        c,
        fname: str,
        bag: Set[str],
        output: wi.Output,
        PX: int = PX,
    ) -> None:
        self.c = c
        self.fname = fname
        self.bag = bag
        self.output = output
        self.PX = PX
        self.id = fname2id(fname)
        self.image = load_wimage(fname, self.PX)
        self.button = wi.Button(description=self.id, layout=wi.Layout(width=f"{PX}px"))
        self.button.on_click(self.on_click)
        super().__init__([self.image, self.button])
class Viewer(wi.Box):
    def __init__(
        self,
        c,
        fnames: Iterable[str],
        N: int = 23,
        seed: int = 0,
        name: str = '',
        key = None,
        PX: int = PX,
        root: Path = root,
    ) -> None:
        self.c = c
        self.N = N
        self.seed = seed
        self.PX = PX
        self.fnames = list(map(str, fnames))
        self.root = root
        if len(self.fnames) == 0:
            super().__init__()
            print("0");  return
        if key == None:
            self.fnames.sort()
            Random(self.seed).shuffle(self.fnames)
        elif key == "nosort":
            pass
        else:
            self.fnames.sort(key=lambda fname: key(fname2id(fname)))
        self.fnames = tuple(self.fnames)
        if name == '':
            self.name = hash(self.fnames)
        else:
            self.name = name
        self.i = 0
        self.bag = set()
        self.select = wi.Select(options=range(len(self)), layout=wi.Layout(width=f"{PX//2}px", height=f"{round(PX)*0.95}px"))
        self.button_n = wi.Button(description="next", layout=wi.Layout(width=f"{PX//2}px"))
        self.button_p = wi.Button(description="prev", layout=wi.Layout(width=f"{PX//2}px"))
        self.button_s = wi.Button(description="save", layout=wi.Layout(width=f"{PX//2}px"))
        self.button_l = wi.Button(description="load", layout=wi.Layout(width=f"{PX//2}px"))
        self.status = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.output = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.controller = wi.VBox([
            wi.HBox([wi.VBox([
                self.button_n,
                self.button_p,
                self.button_s,
                self.button_l,
            ]), self.select]),
            self.status,
            self.output,
        ])
        self.layout = wi.Layout(flex_flow="row wrap")
        self.button_n.on_click(self.on_click_n)
        self.button_p.on_click(self.on_click_p)
        self.button_s.on_click(self.save)
        self.button_l.on_click(self.load)
        self.select.observe(self.on_select, "value")
        super().__init__()
        if self.is_saved():
            self.load()
        else:
            self.refresh()
    def __len__(self) -> int:
        return (len(self.fnames)-1)//self.N+1
    def __getitem__(self, i: int) -> List[Pannel]:
        assert i in range(len(self))
        return [
            Pannel(self.c, fname, self.bag, self.output, self.PX)
            for fname in self.fnames[i*self.N:(i+1)*self.N]
        ]
    def refresh(self) -> None:
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames))
        self.children = tuple(self[self.i]+[self.controller])
    def on_click_p(self, event) -> None:
        self.i -= 1; self.i %= len(self);
        self.select.value = self.i
    def on_click_n(self, event) -> None:
        self.i += 1; self.i %= len(self);
        self.select.value = self.i
    def on_select(self, event) -> None:
        self.i = event["new"]; self.refresh()
    def is_saved(self) -> bool:
        return  (self.root/"viewer"/f"{self.name}.pkl").is_file()
    def save(self, event=None) -> None:
        self.status.clear_output()
        with self.status:
            print("saving...")
        (self.root/"viewer").mkdir(exist_ok=True)
        with open(self.root/"viewer"/f"{self.name}.pkl", "wb") as f:
            pickle.dump(self.bag, f, protocol=4)
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames), "saved")
    def load(self, event=None) -> None:
        if not self.is_saved():
            self.status.clear_output()
            with self.status:
                print(f"{self.i}/{len(self)}", len(self.fnames), "not saved")
            return
        self.status.clear_output()
        with self.status:
            print("loading...")
        with open(self.root/"viewer"/f"{self.name}.pkl", "rb") as f:
            self.bag = pickle.load(f)
        self.refresh()
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames), "loaded")



In [ ]:
"abc.npz"[-4:]

In [ ]:
ID, canvas = spylus.ID_multicanvas()
marker = HTML(f'<div id="{ID}"></div>')
selector = wi.Dropdown(description="レイヤー番号", options=range(2), value=1)

reload_button0 = wi.Button(description="reload", layout=wi.Layout(width="130px"))

save_input1 = wi.Text(layout=wi.Layout(width="30px"))
copy_button1 = wi.Button(description="コピー", layout=wi.Layout(width="100px"))
paste_button1 = wi.Button(description="ペースト", layout=wi.Layout(width="100px"))
clear_button1 = wi.Button(description="全消去", layout=wi.Layout(width="100px"))
color_input1 = wi.ColorPicker(description="線色", value='#000000', layout=wi.Layout(width="200px"))
width_select1 = wi.Dropdown(description="線幅", options=range(2,41,2), value=2, layout=wi.Layout(width="200px"))
seed_input1 = wi.Text(description="塗り方のseed", value="ランダム", layout=wi.Layout(width="300px"))
toggle_button1 = wi.ToggleButton(description="消しゴム", value=True, button_style='success')
out = wi.Output()
fc0 = FileChooser("/home/natsuki/pixel2style2pixel/examples/sim")

    
@out.capture(clear_output=True)
def qrc(fname):
    img = Image.open(fname)
    try:
        if pyzbar.decode(img)[0].data == b'/home/natsuki/mao_DBmiyuki/maoPR1.npz':
            seed_input1.value = '/home/natsuki/mao_DBmiyuki/maoPR1.npz'
            return
    except:
        pass
    try:
        print(pyzbar.decode(img)[0].data)
        seed = int(pyzbar.decode(img)[0].data)
    except:
        seed = 0
    if seed == 0:
        seed_input1.value = "ランダム"
    else:
        seed_input1.value = str(seed)
        
@out.capture(clear_output=True)
def simc(fname):
    image = Image.open(fname)
    text = spylus.encode(image)
    display(Javascript(f'app1.load("{text}");'))
qrs = list(map(str, Path("/home/natsuki/pixel2style2pixel/examples/qr").glob("*.png")))
qrs.sort()
qrs = ["/home/natsuki/mao_DBmiyuki/maoQR1.png"]+qrs
qrv = Viewer(qrc, qrs, N=7+6, name="qrs", key ="nosort")
sims = list(map(str, Path("/home/natsuki/pixel2style2pixel/examples/sim").glob("*.png")))
sims.sort()
maos = list(map(str, Path("/home/natsuki/mao_DBmiyuki").glob("maoSM*.png")))
sims = maos+sims
simv = Viewer(simc, sims, N=7+6, name="sims", key ="nosort")

@out.capture(clear_output=True)
def load0(chooser):
    image = Image.open(fc0.selected)
    text = spylus.encode(image)
    display(Javascript(f'app1.load("{text}");'))
fc0.register_callback(load0)

@out.capture(clear_output=True)
def paste_button1_on_click(b):
    print("クリスタのレイヤーを選択してコピーして貼り付けてください")
    print("そのほかのデータ形式だと失敗することがあるので上記手順を推奨です")
paste_button1.on_click(paste_button1_on_click)

@out.capture(clear_output=True)
def copy_button1_on_click(b):
    print("ブラウザに聞かれたら「許可」を押すことで以降クリップボードにコピーされます")
    print("クリスタのレイヤーに貼り付けることができます")
copy_button1.on_click(copy_button1_on_click)

@out.capture(clear_output=True)
def on_toggle(change):
    if not change["new"]:
        print("消しゴムに切り替わりました")
        color_input1.value = "#FFFFFF"
        width_select1.value = 10
    else:
        print("えんぴつに切り替わりました")
        color_input1.value = "#000000"
        width_select1.value = 2
    globals()["change"] = change
toggle_button1.observe(on_toggle, "value")


out_ca = ca.Canvas(width=512, height=512)
out_ca.sync_image_data = True
out_ca.fill_style = '#a9cafc'
out_ca.fill_rect(0, 0, 512, 512)
out_ca.fill_style = 'black'
out_ca.font = '32px serif'
#out_ca.fill_text('Initialized', 0, 512)

synth_button = wi.Button(description="画像生成！", layout=wi.Layout(width="200px"))
@out.capture(clear_output=True)
def synth(b, filter=Image.BICUBIC):
    out_ca.fill_text('Synthesizing...', 0, 512)
    sketch_pillow = spylus.decode(save_input1.value).resize((256, 256), filter)
    sketch_numpy =  np.array(sketch_pillow, dtype='float32')
    sketch_torch = torch.from_numpy(sketch_numpy.transpose((2,0,1))[None,:1,:,:]/255).cuda() #Rのみ
    try:
        seed = int(seed_input1.value)
        if seed == 0:
            latent_to_inject = None
        else:
            latent_to_inject = mapping(net.decoder.G, seed)
    except:
        try:
            with np.load(seed_input1.value) as data:
                latent_to_inject = torch.from_numpy(data["w"]).cuda()
                print("ようこそ!", latent_to_inject.shape)
        except:
            latent_to_inject = None
    output_torch = run_on_batch(sketch_torch, net, opts, latent_to_inject)
    output_pillow = common.tensor2im(output_torch[0])
    output_numpy = np.array(output_pillow)
    out_ca.put_image_data(output_numpy)
synth_button.on_click(synth)

@out.capture(clear_output=True)
def on_mouse_down(x, y):
    color_input1.value = '#'+''.join(map(lambda c: hex(c)[2:], out_ca.get_image_data()[int(y),int(x)][0:3]))
out_ca.on_mouse_down(on_mouse_down)

reload_button0.on_click(load0)

app = wi.VBox((
    wi.HBox((out_ca, canvas,)),
    wi.HBox((save_input1, copy_button1, paste_button1, clear_button1, color_input1, width_select1,toggle_button1,)),
    wi.HBox((selector, seed_input1, synth_button)),
    qrv,
    simv,
#    fc0,
    out,
))
display(marker, app)

jscode = Javascript(f"""{spylus.js}
window.app1 = new (mix(
    White, Save, Load, Copy, Paste, Color, Width, Clear
))({{
    canvas: document.getElementById("{ID}1"),
    save_input: elem('{ID}', 'input', 0),
    copy_button: elem('{ID}', 'button', 0),
    paste_button: elem('{ID}', 'button', 1),
    clear_button: elem('{ID}', 'button', 2),
    color_input: elem('{ID}', 'input', 1),
    width_select: elem('{ID}', 'select', 0),
}});

new Selector({{
    apps: [app1, app1],
    selector_select: elem('{ID}', 'select', 1),
}});
""")
with out:
    display(jscode)